In [ ]:
### 적당한 문장을 남기고 필터링하여 품사 태깅을 수행합니다. => comment_corpus.tag

from konlpy.tag import Mecab
from konlpy.utils import pprint
import os
from sgmllib import SGMLParser, SGMLParseError
import sys

class TextExtracter(SGMLParser):
    """
    HTML 문서에서 태그와 엔티티를 제거하고 텍스트만 추출하는 클래스
    """
    def __init__(self):
        self.text = []
        SGMLParser.__init__(self)
    def handle_data(self, data):
        self.text.append(data)
    def getvalue(self):
        return ''.join(self.text)
    @classmethod
    def get_text(cls, html):
        txt_ext = TextExtracter()
        txt_ext.feed(html)
        return txt_ext.getvalue()

def ascii_count(text):
    """
    문자열에서 ASCII 영역 문자의 갯수를 세는 함수
    @param  text  문자열
    @return       ASCII 문자 갯수
    """
    return sum([1 for char in text if ord(char) & 0x80 == 0])

mecab = Mecab('%s/usr/mecab-ko/lib/mecab/dic/mecab-ko-dic' % os.environ['HOME'])

with open('comment_corpus.tag', 'w') as fout:
    for line_num, line in enumerate(open('comment_corpus.txt'), start=1):    # 댓글 코퍼스를 읽어들입니다.
        line = line.strip()
        if not line:
            continue
        try:
            line = TextExtracter.get_text(line)    # 태그를 제거하고 텍스트만 남깁니다.
        except SGMLParseError:
            pass
        if ascii_count(line) > (len(line) / 5):    # ASCII 문자의 갯수가 전체의 1/5을 넘는 댓글은 제거합니다.
            continue
        words = line.split()
        if len(words) < 5:    # 단어의 갯수가 5개 미만인 댓글은 제거합니다.
            continue
        raw = ' '.join(words)
        tagged = ['%s/%s' % (morph.encode('UTF-8'), tag.encode('UTF-8'))    # '형태소/태그' 형태로 저장합니다.
                  for morph, tag in mecab.pos(unicode(raw, 'UTF-8'))]
        print >> fout, ' '.join(tagged)

!head -n5 comment_corpus.tag

In [ ]:
### 200만 문장을 샘플링하여 gensim을 이용해 doc2vec을 학습합니다. => comment_corpus_2m.d2v

!shuf -n 2000000 comment_corpus.tag > comment_corpus.tag.2m

import gensim
import sys

path = 'comment_corpus_2m.tag'
def documents():
    """
    전체 문서를 읽어들여 문장(LabeledSentence 객체)을 하나씩 반환하는 제너레이터
    """
    print >> sys.stderr, path
    for line_num, line in enumerate(open(path), start=1):
        if line_num % 1000000 == 0:
            print >> sys.stderr, '    %dm-th line' % (line_num / 1000000)
        if not line:
            continue
        words = [unicode(word, 'UTF-8') for word in line.split()]    # 단어는 모두 유니코드로 저장해야 합니다.
        yield gensim.models.doc2vec.LabeledSentence(words=words, tags=[unicode(line_num),])

d2v_model = gensim.models.Doc2Vec(size=300, alpha=0.025, min_alpha=0.025, workers=20)
d2v_model.build_vocab(documents())    # vocabulary를 먼저 생성합니다.
for epoch in range(10):    # 10회 반복하여 학습을 진행합니다.
    print >> sys.stderr, '[%d] epoch:' % epoch,
    d2v_model.train(documents())
    d2v_model.alpha -= 0.002    # alpha 값을 줄여 learning rate를 서서히 줄입니다.
    d2v_model.min_alpha = d2v_model.alpha    # epoch 내에서는 learning rate를 decay 없이 고정합니다.
d2v_model.save(path.replace('.tag', '.d2v'))